<!--
SPDX-FileCopyrightText: 2025 Silva F. F. <fernandoferreira.silva42@usp.br>
SPDX-FileCopyrightText: 2025 Monaco F. J. <monaco@usp.br>

SPDX-License-Identifier: GPL-3.0-or-later
-->

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opensciware/moeabench/blob/main/examples/example-06.ipynb)

# MoeaBench Example 06: Statistical Analysis ("Smart Stats")

This example demonstrates how to perform a rigorous statistical comparison between two algorithms using the **"Smart Stats"** API. We will compare `NSGA-III` and `SPEA2` on the `DTLZ2` problem, automatically handling metric calculation and fair reference points.

In [ ]:
# Install MoeaBench
!pip install --quiet git+https://github.com/opensciware/moeabench.git

In [ ]:
import mb_path
from MoeaBench import mb
import numpy as np

# 1. Setup Algorithm Comparison
repeats = 10
pop_size = 100
gens = 50

print(f"Running {repeats} repetitions for NSGA3 and SPEA2...")

# Experiment 1: NSGA3
exp1 = mb.experiment()
exp1.name = "NSGA3"
exp1.mop = mb.mops.DTLZ2(M=3)
exp1.moea = mb.moeas.NSGA3(population=pop_size, generations=gens)
exp1.run(repeat=repeats, workers=-1)

# Experiment 2: SPEA2
exp2 = mb.experiment()
exp2.name = "SPEA2"
exp2.mop = mb.mops.DTLZ2(M=3)
exp2.moea = mb.moeas.SPEA2(population=pop_size, generations=gens)
exp2.run(repeat=repeats, workers=-1)

In [ ]:
# 2. Statistical Analysis ("Smart Stats")
print("--- Statistical Tests ---")

# Mann-Whitney U Test (Significance)
# By default, it uses mb.hv and extracts the final generation.
res = mb.stats.mann_whitney(exp1, exp2)
p_value = res.pvalue

print(f"Mann-Whitney U P-value: {p_value:.2e}")
if p_value < 0.05:
    print("  -> Significant Difference (p < 0.05)!")
else:
    print("  -> No Significant Difference.")

# Kolmogorov-Smirnov Test (Distribution Shape)
ks_res = mb.stats.ks_test(exp1, exp2)
print(f"Kolmogorov-Smirnov P-value: {ks_res.pvalue:.2e}")
if ks_res.pvalue < 0.05:
    print("  -> Distributions have different shapes/stabilities!")
    
# Vargha-Delaney A12 (Effect Size)
# Measures probability that 1 > 2.
effect_size = mb.stats.a12(exp1, exp2)
print(f"Vargha-Delaney A12 (1 vs 2): {effect_size:.4f}")

if effect_size == 0.5:
    print("  -> Magnitude: Negligible")
elif effect_size > 0.5:
    print(f"  -> {exp1.name} is better than {exp2.name}")
else:
    print(f"  -> {exp2.name} is better than {exp1.name}")